# Fake News Detector

### Installing Necessary Libraries

In [29]:
!pip install farasa
import re
import string
from farasa.segmenter import FarasaSegmenter
from farasa.stemmer import FarasaStemmer
from farasa.pos import FarasaPOSTagger


### Loading the data

In [20]:
import pandas as pd

# Load dataset
path = './data_set/fake_news_dataset.csv'
fake_data = pd.read_csv(path, header=None,names=['text_column'])

if fake_data.empty:
    raise ValueError("The DataFrame is empty. Please check the CSV file ❌")
else :
    print("DataFrame loaded successfully ✅")


DataFrame loaded successfully ✅


### Data Preview

In [21]:
fake_data.head()

,text_column
0,اكتشاف علاج نهائي لجميع أنواع السرطان خلال أسبوع!
1,حكومة دولة عربية تعلن عن توزيع الأموال مجانًا ...
2,رجل يشرب ماءً فقط لمدة 365 يومًا ويبقى بصحة جيدة!
3,علماء يؤكدون أن الأرض مسطحة وليست كروية!
4,إغلاق جميع المدارس في البلاد بسبب تغير المناخ!


In [22]:
fake_data.shape

(10, 1)

In [23]:
fake_data.columns

Index(['text_column'], dtype='object')

### Preprocessing Text


In [51]:
def clean_text(text):
    """Basic text cleaning for Arabic"""
    if not text or not isinstance(text, str):
            return ""
    # Remove URLs
    text = re.sub(r'https?://\S+|www\.\S+', '', text)
    
    # Remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # Remove diacritics (tashkeel)
    text = re.sub(r'[\u064B-\u065F\u0670]', '', text)
    
    # Normalize Alef variations to Alef
    text = re.sub(r'[إأآا]', 'ا', text)
    
    # Normalize Ya and Alef Maksura
    text = re.sub(r'[يى]', 'ي', text)
    
    # Normalize Hamzas
    text = re.sub(r'[ؤئ]', 'ء', text)
    
    # Remove non-Arabic characters except spaces and numbers
    text = re.sub(r'[^\u0600-\u06FF\s0-9]', '', text)
    
    # Remove extra spaces
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

### Text Cleaning and Normalization (Using Farasa)

In [52]:

# Initialize Farasa tools
segmenter = FarasaSegmenter()
stemmer = FarasaStemmer()
pos_tagger = FarasaPOSTagger()

# You can also load a stopwords list from a file if you have one.
with open('./data_set/list.txt', "r", encoding="utf-8") as f:
    ARABIC_STOPWORDS = set(f.read().splitlines())

def clean_text(text):
    # Normalize Alif (أ, إ, آ → ا) and Ta Marbuta (ة → ه)
    text = text.replace("أ", "ا").replace("إ", "ا").replace("آ", "ا").replace("ة", "ه")
    # Tokenize & Stem using Farasa
    tokens = segmenter.segment(text).split()
    stemmed_tokens = [stemmer.stem(word) for word in tokens]

    # Remove stopwords
    final_tokens = [word for word in stemmed_tokens if word not in ARABIC_STOPWORDS]
    
    return " ".join(final_tokens)

# Apply preprocessing to dataset
fake_data["cleaned_text"] = fake_data["text_column"].apply(clean_text)

# Save preprocessed dataset
fake_data.to_csv("preprocessed_dataset.csv", index=False)

print(fake_data.head())


                                         text_column  \
0  اكتشاف علاج نهائي لجميع أنواع السرطان خلال أسبوع    
1  حكومة دولة عربية تعلن عن توزيع الأموال مجان ا ...   
2       رجل يشرب ماء فقط لمدة يوم ا ويبقى بصحة جيدة    
3           علماء يؤكدون أن الأرض مسطحة وليست كروية    
4     إغلاق جميع المدارس في البلاد بسبب تغير المناخ    

                                        cleaned_text  
0      اكتشاف علاج نهائي ل  جميع نوع ال  سرطان أسبوع  
1  حكوم  ه دولة  ه عربي  ه أعلن توزيع ال  مال مجا...  
2        رجل شرب ماء ل  مد  ه و  بقي ب  صح  ه جيد  ه  
3     عالم أكد  ون ال  أرض مسطح  ه و  ليس  ت كروي  ه  
4       إغلاق ال  مدرسة ال  بلد ب  سبب تغير ال  مناخ  


###  analyse morphologique With Farasa

In [ ]:
import pandas as pd
from farasa.pos import FarasaPOSTagger

# Initialize Farasa POS Tagger
farasa_pos = FarasaPOSTagger()

# Load preprocessed dataset
df_cleaned = pd.read_csv("./preprocessed_dataset.csv")

# Apply POS tagging to each sentence
df_cleaned["pos_tagged"] = df_cleaned["cleaned_text"].apply(lambda text: farasa_pos.tag(text))

# Print results in a readable format
for i, sentence in enumerate(df_cleaned["pos_tagged"]):
    print(f"🔹 Sentence {i+1}:")
    for item in sentence:
        if "/" in item:  # Ensure correct format
            word, tag = item.rsplit("/", 1)  # Split only on the last "/"
            print(f"{word} → {tag}")
    print("\n" + "-" * 50 + "\n")  # Add a separator between sentences

#  Convert TF-IDF Matrix to a DataFrame

## Apply Machine Learning (Optional)